#### MATH 448 Project Report 1 (EDA): 
#### Understanding Public Data and Predict Probability of Success of Newly Opening Business.


* Data Extraction

* Data Preperation

* Data Visualization


In [2]:
# Requring Library
import requests
import pandas as pd
import pprint

from api_key import API_key

In [3]:
# Read csv file
public_data = pd.read_csv("data/November_2014_Street_Cleaning_TTC_20240213.csv")

public_data.head()

,CaseID,Opened,Closed,Updated,Status,Responsible Agency,Category,Request Type,Request Details,Address,Supervisor District,Neighborhood,Point,Source,Media URL
0,4165985,11/01/2014 12:06:00 AM,11/01/2014 12:58:50 AM,11/01/2014 12:58:50 AM,Closed,Recology_Overflowing,Street and Sidewalk Cleaning,Overflowing_City_Receptacle_or_Dumpster,Overflowing_City_Receptacle_or_Dumpster,Intersection of PINE ST and FRONT ST,3.0,Financial District,"(37.79240799, -122.39859009)",Integrated Agency,NaN
1,4165986,11/01/2014 12:06:03 AM,11/01/2014 06:40:33 AM,11/01/2014 06:40:33 AM,Closed,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Refrigerator,"4340 MISSION ST, SAN FRANCISCO, CA, 94112",11.0,Mission Terrace,"(37.72815704, -122.43209076)",Integrated Agency,NaN
2,4166013,11/01/2014 12:47:19 AM,11/01/2014 10:00:31 AM,11/01/2014 10:00:31 AM,Closed,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Refrigerator,"4340 MISSION ST, SAN FRANCISCO, CA, 94112",11.0,Mission Terrace,"(37.72815704, -122.43209076)",Integrated Agency,NaN
3,4166014,11/01/2014 12:47:21 AM,11/01/2014 12:59:28 AM,11/01/2014 12:59:28 AM,Closed,Recology_Overflowing,Street and Sidewalk Cleaning,Overflowing_City_Receptacle_or_Dumpster,Overflowing_City_Receptacle_or_Dumpster,Intersection of PINE ST and FRONT ST,3.0,Financial District,"(37.79240799, -122.39859009)",Integrated Agency,NaN
4,4166015,11/01/2014 12:47:24 AM,11/01/2014 06:44:04 AM,11/01/2014 06:44:04 AM,Closed,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Refrigerator,Intersection of LEAVENWORTH ST and EDDY ST,5.0,Tenderloin,"(37.78372192, -122.41423798)",Integrated Agency,NaN


In [7]:
# Check columns and data type
public_data.dtypes

CaseID                   int64
Opened                  object
Closed                  object
Updated                 object
Status                  object
Responsible Agency      object
Category                object
Request Type            object
Request Details         object
Address                 object
Supervisor District    float64
Neighborhood            object
Point                   object
Source                  object
Media URL               object
dtype: object

In [16]:
# Set up Yelp API credentials
headers = {
    'Authorization': f'Bearer {API_key}'
}

# Set up the search parameters
search_term = 'shoe cleaning'
location = 'San Francisco'  # You can change the location as per your preference

# Set up the Yelp API endpoint for business search
url = 'https://api.yelp.com/v3/businesses/search'
params = {
    'term': search_term,
    'location': location
}

# Make the API request
response = requests.get(url, headers=headers, params=params)
data = response.json()


In [17]:
pprint.pprint(data)

{'businesses': [{'alias': 'model-shoe-renew-berkeley',
                 'categories': [{'alias': 'shoerepair', 'title': 'Shoe Repair'},
                                {'alias': 'shoes', 'title': 'Shoe Stores'}],
                 'coordinates': {'latitude': 37.873255,
                                 'longitude': -122.2685785},
                 'display_phone': '(510) 843-7218',
                 'distance': 19326.665498026163,
                 'id': 't_JlTzyXMOvNl7gIWxAWzw',
                 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/0-gTVnGnUBi5XZtmYZIgZw/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '1934 Shattuck Ave',
                              'address2': '',
                              'address3': '',
                              'city': 'Berkeley',
                              'country': 'US',
                              'display_address': ['1934 Shattuck Ave',
                                                  'Berkele

In [ ]:
# Extract relevant information from the response
businesses = data['businesses']


